## La librería re - Expresiones regulares

La librería **`re`** permite trabajar con expresiones regulares.

Vamos a realizar primero un ejercicio para entender la necesidad y utilidad de las expresiones 
regulares. Vamos a hacer una funcion que nos permita identificar si un texto representa un número
de teléfono del tipo `999-999-9999`, como por ejemplo `354-472-5723`. 

In [3]:
import sys

print(sys.base_prefix, sys.version)

/usr 3.8.5 (default, Jul 28 2020, 12:59:40) 
[GCC 9.3.0]


In [2]:
import ipytest
ipytest.autoconfig()

In [3]:
%%run_pytest[clean] -qq

def is_phone(s):
    pass

def test_ok():
    assert is_phone('354-472-4237') is True

def test_bad():
    assert is_phone('tururu') is False
    assert is_phone('tururutururu') is False
    assert is_phone('tur-rut-ruru') is False
    assert is_phone('123-rut-ruru') is False
    assert is_phone('123-456-ruru') is False
    assert is_phone('tur-456-9876') is False

FF                                                                                                                            [100%]
============================================================= FAILURES ==============================================================
______________________________________________________________ test_ok ______________________________________________________________

    def test_ok():
>       assert is_phone('354-472-4237') is True
E       AssertionError: assert None is True
E        +  where None = is_phone('354-472-4237')

<ipython-input-3-17d950e7f031>:5: AssertionError
_____________________________________________________________ test_bad ______________________________________________________________

    def test_bad():
>       assert is_phone('tururu') is False
E       AssertionError: assert None is False
E        +  where None = is_phone('tururu')

<ipython-input-3-17d950e7f031>:8: AssertionError
==================================================

Veamos una primera implementacion:

In [47]:
%%run_pytest[clean] -qq -v

def is_phone(s):
    if len(s) != 12:
        return False
    if not s[:3].isdigit():
        return False
    if s[3] != '-':
        return False
    if not s[4:7].isdigit():
        return False
    if s[7] != '-':
        return False
    if not s[8:].isdigit():
        return False
    return True


def test_ok():
    assert is_phone('354-472-4237') is True

def test_bad():
    assert is_phone('tururu') is False
    assert is_phone('tururutururu') is False
    assert is_phone('tur-rut-ruru') is False
    assert is_phone('123-rut-ruru') is False
    assert is_phone('123-456-ruru') is False
    assert is_phone('tur-456-9876') is False
    assert is_phone('123-fur-9876') is False

..                                                                                                                            [100%]


Bien, todos los test pasas. Pero... ¿qué pasa ahora si queremos que se acepten dos formatos,
tanto `354-472-4237` como `(354) 472-4237`? Vamos primero a añadir un test para este nuevo caso:

In [48]:
%%run_pytest[clean] -qq -v

def test_ok_alt_format():
    assert is_phone('(354) 472-4237') is True

F                                                                                                                             [100%]
============================================================= FAILURES ==============================================================
________________________________________________________ test_ok_alt_format _________________________________________________________

    def test_ok_alt_format():
>       assert is_phone('(354) 472-4237') is True
E       AssertionError: assert False is True
E        +  where False = is_phone('(354) 472-4237')

<ipython-input-48-a0b4b14b5e70>:2: AssertionError
====================================================== short test summary info ======================================================
FAILED tmpewhvc95q.py::test_ok_alt_format - AssertionError: assert False is True


Obviamente, falla. Vamos a inplementar una version mejorada que arregle este test.

In [60]:
'(354) 472-4237'[10:]

'4237'

In [61]:
%%run_pytest[clean] -qq -v

def is_phone(s):
    if len(s) == 12:
        if not s[:3].isdigit():
            return False
        if s[3] != '-':
            return False
        if not s[4:7].isdigit():
            return False
        if s[7] != '-':
            return False
        if not s[8:].isdigit():
            return False
        return True
    elif len(s) == 14:
        if s[0] != '(':
            return False
        if not s[1:4].isdigit():
            return False
        if s[4] != ')':
            return False
        if s[5] != ' ':
            return False
        if not s[6:9].isdigit():
            return False
        if s[9] != '-':
            return False
        if not s[10:].isdigit():
            return False
        return True
    return False

def test_ok():
    assert is_phone('354-472-4237') is True
    
def test_ok_alt_format():
    assert is_phone('(354) 472-4237') is True
    
def test_bad():
    assert is_phone('tururu') is False
    assert is_phone('tururutururu') is False
    assert is_phone('tur-rut-ruru') is False
    assert is_phone('123-rut-ruru') is False
    assert is_phone('123-456-ruru') is False
    assert is_phone('tur-456-9876') is False
    assert is_phone('123-fur-9876') is False

...                                                                                                                           [100%]


In [8]:
%%run_pytest[clean] -qq -v

def is_phone(s):
    pat_telefono = re.compile(r'\d{3}-\d{3}-\d{4}|\(\d{3}\) \d{3}-\d{4}')
    if pat_telefono.match(s):
        return True
    return False
    

def test_ok():
    assert is_phone('354-472-4237') is True
    
def test_ok_alt_format():
    assert is_phone('(354) 472-4237') is True
    
def test_bad():
    assert is_phone('tururu') is False
    assert is_phone('tururutururu') is False
    assert is_phone('tur-rut-ruru') is False
    assert is_phone('123-rut-ruru') is False
    assert is_phone('123-456-ruru') is False
    assert is_phone('tur-456-9876') is False
    assert is_phone('123-fur-9876') is False

...                                                                                                                           [100%]


Ya funciona con el nuevo formato. Pero es un montón de código, yno especialmente sencillo (demasiados if y niveles de anidamiento). Es verdad que se podria haber escrito de otra forma más corta, pero no mucho más. Veremos ahora qué son las expresiones regulares y como podemos usarlas para reimplementar todo el codigo de la funcion en apenas dos o tres líneas.

### Qué son las expresiones regulares

Una **expresión regular** viene a definir un conjunto de cadenas de texto que
cumplen un determinado patrón. Si una cadena de texto pertenece al conjunto de
posibles cadenas definidas por la expresión, se dice que *casan* o que ha habido
una coincidencia o *match*.

Existen múltiples implementaciones de expresiones regulares, cada una compartiendo
un nucleo comun pero con extensiones o modificaciones para añadir capacidades más
avanzadas.

La sintaxis usada en el módulo `re` de python se basa en la usada por
el lenguaje Perl, con unas pocas mejoras especificas de Python.

Las expresiones regulares se crean combinando expresiones regulares más
pequeñas (o primitivas), y se especifican mediante una cadena de texto.

La cadena que define una expresión regular puede incluir caractereres
__normales__ o __especiales__. 

Los caracteres normales solo casan consigo mismo. Por ejemplo, la expresión
regular `a` solo casaría con una a.

Los especiales, como `|` o `.` tienen otros significados; o bien definen
conjuntos de caracteres o modifican a las expresiones regulares adyacentes.

Antes de ver los caracteres especiales, veamos como usar las expresiones regulares. Primero tenemos que importar el modulo `re`:

In [35]:
import re

El siguiente ejemplo usa una expresión regular para encontrar números dentro de una
cadena de texto. No hay que preocuparse ahora de lo que significa la expresión 
regular `\d+`, eso lo explicaremos en el resto del tema. Ahora solo hay que entender
como se usan.

Primero, a partir del texto que describe el patron que queremos buscar, `\d+` en este
caso, l;lamando a la función `compile` del módulo `re`, se obtiene un objeto tipo
_pattern_ (Patrón). Este objeto está especializado en identificar el patron que le
hemos pasado. Los objetos _pattern_ tienen varios métodos útiles, vamos a
usar `search`, que realiza una búsqueda del patrón en el texto indicado. Si
no lo encuentra, devuelve `None`, pero si lo encuentra, devuelve un objeto
de tipo `match`, que, entre otras cosas, nos indica donde
exactamente dentro del texto se encuentra el subtexto que casa con el patrón:


In [64]:
import re 

patron = re.compile(r"\d+")

s = "Con 100 cañones por banda..."
match = patron.search(s)
if match:
    print('Encontrado', match)
else:
    print('No encontrado')

Encontrado <re.Match object; span=(4, 7), match='100'>


**Nota**: No es estríctamente obligatorio usar la función `compile`.
Podemos usar una función `search`, definida en `re`, a la que le pasamos dos
parámetros, el primero la cadena de texto que describe la expresión
regular, y el segundo el texto a buscar. Internamente la función creara el objeto
patron. Yo recoiendo compilar primero, porque es más eficiente y de esa forma podemos reutilizar  el patrón en diferentes sitios. Este seria el codigo del ejemplo usando la función `search` en vez del método `search`:

In [65]:
import re

s = "Con 100 cañones por banda..."
match = re.search(r"\d+", s)
if match:
    print('Encontrado', match)
else:
    print('No encontrado')

Encontrado <re.Match object; span=(4, 7), match='100'>


### Resultado de buscar patrones en un texto

COmo se explica antes, el resultado de la función depende de si ha encontrado
o no alguna ocurrencia del patrón en el texto. Si la encontró, se devuelve
un objeto de tipo `Match` (que es un objeto que almacena la información de donde se ha encotrado). Si no lo encuentra, devuelve `None`.

Entra la información que podemos encontrar en este objeto `Match`, se incluye el texto
que ha encontrado, la expresion regular usada y la localización, dentro del texto buscado, de
esa coincidencia.

Veamos un ejemplo, usando como expresion regular `este`. Como en esta expresion solo hay caracteres
normales, se interpreta como: "Una `e`, seguida de una `s`, seguida de una `t`, seguida de una `e`:

In [87]:
import re

pattern = re.compile(r'este')
text = 'Contiene este texto el patrón?'
match = pattern.search(text)
if match:
    print(
        f"Encontrado <{match.group(0)}>"
        f" entre las posiciones {match.start()}"
        f" y {match.end()}"
    )

Encontrado <este> entre las posiciones 9 y 13


**Miniejercicio**: Modificar el código previo para que encuentr la palabra "texto"

En principio, nada que no pudieramos hacer usando el método `index` de las cadenas de texto. La potencia de las expresiones regulares viene de los caracteres especiales.

Algunos caracteres con significados especiales son:

| Caracter | Casa con                                                        |
|---------:|-----------------------------------------------------------------|
| `.`      | Cualquier caracter                                              |
| `^`      | El principio de una string                                      | 
| `$`      | El final de una string                                          |
| `*`      | La expresión regular anterior, repetida 0 o más veces           |
| `+`      | La expresión regular anterior, repetida 1 o más veces           |
| `?`      | La expresión regular anterior, 0 o 1 vez                        |
| `{n}`    | La expresión regular anterior, repetida n veces                 |
| `{m,n}`  | La expresión regular anterior, repetida entre m y n veces       |
| `\`      | "Escapa" el significado del caracter a continuación             |
| `|`      | Alernancia entre patrones: `A|B` casa con A o con B             |
| `[...]`  | El conjunto de caracteres definido entre los corchetes          |

Los veremos ahora con más detalle.

### El caracter especial punto `.` 

El punto es un caracter especial, por lo que tiene un significado diferente
de "debe ser un punto". En una expresion regular, el carácter `.` significa
"cualquier caracter", es decir, es un comodín, para caracteres. Pero atención, que 
solo _casa_ con un unico caracter. 

Por ejemplo, el patrón regular `est.` casaria con `este`, `esta`, `estx`, `est8`, `est@`, pero __no__ con `est`, porque espera un cuarto caracter, el que eea, pero no encuentra ninguno.

In [89]:
import re

pattern = re.compile('te.to')
text = 'Contiene este texto el patrón?'

match = pattern.search(text)
if match:
    print(
        f"Encontrado <{match.group(0)}>"
        f" entre las posiciones {match.start()}"
        f" y {match.end()}"
    )

Encontrado <texto> entre las posiciones 14 y 19


**Miniejercicio**

1) Cambiar la variable `text` del ejericio anterior por "`Contiene este teZto el patrón?`". Verificar que sigue encontrado el patrón.

2) Cambiar la variable `text` por "`Contiene este teZXto el patrón?`". ¿Encuentra ahora el patrón? ¿Por qué?


### El caracter especial Acento Circunflejo `^`

El caracter especial `^` se interpreta como "Al principio del texto". Sirve para buscar textos que empiezan por la expresion regular que venga despues. 

Por ejemplo, el patrón regular "`^Carthago`" solo casaría con un texto que *empiece* con la palabra `Carthago`.

**Pregunta**: Tiene sentido que el caracter especial `^` se use en una expresión regular en otro sitio que no sea al principio? ¿Por qué?

### El caracter especial Dolar `$`

Ser'ia el inverso del anterior, este caracter especial se interpreta como "Al final del texto". Sirve para buscar textos que termina por la expresion regular que viene justo antes. 

Por ejemplo, el patrón regular "`Delenda est$`" solo casaría con un texto que __termine__ con las palabras `Delenda est`.

**Pregunta**: Tiene sentido que el caracter especial `$` se use en una expresión regular en otro sitio que no sea al final? ¿Por qué?

### El caracter especial Asterisco `*`

El caracter especial `*` debe interpretarse como "El patron anterior, repetido 0 o más veces". Por ejemplo, el
patron `e*` casaria con la cadena vacia (Ninguna aparición del caracter `e`), con `e` (Una repetición del caracter 'e'), con `ee` (Dos repeticiones), `eee` (Tres repeticiones), etc.

Una combinacion muy habitual es el patrón `.*`. Esto se interpreta como "Cero o más repeticiones de la expresion regular que esta justo antes, que en este caso es _cualquier caracter_", o lo que es lo mismo, "cualquier caracter, repetido 0 o más veces". Aun más resumido: "Todo".

In [82]:
p = re.compile("<.*>")
s = "<p>Hola mundo soy <b>pepe Garcia</b> tu amigo</p>"
p.findall(s)



['<p>Hola mundo soy <b>pepe Garcia</b> tu amigo</p>']

Por ejemplo, la expresion regular "`BEGIN .* END`" sería: "Todo lo que haya entre la palabra `BEGIN ` (en mayúsculas y con un espacio, ojo a eso) y "` END`". 

In [91]:
import re

pattern = re.compile(r'BEGIN .* END')
text = 'BEGIN Cualquier cosa que pongamos aqui vale END'

match = pattern.search(text)
if match:
    print(
        f"Encontrado <{match.group(0)}>"
        f" entre las posiciones {match.start()}"
        f" y {match.end()}"
    )

Encontrado <BEGIN Cualquier cosa que pongamos aqui vale END> entre las posiciones 0 y 47


**Miniejercicio** Cambiar el texto entre BEGIN y END. Ver que cualquier cosa que ponemos vale

COn este operador y con el siguiente se produce una ambigüedad, que veremos con
un ejemplo. Supongamos que quiero todo el texto comprendido entre los caracteres `<` y ,`>`. La expresión regular sería `<.*>`, hasta aqui todo bien. Pero que pasa si buscamos ese patron en el siguiente texto:

    Hola <empieza aqui pero termina en el primer > o en el último >
    
¿Debería devolver el fragmento de texto más pequeño:

    `<empieza aqui pero termina en el primer >`
    
O el más grande?:

    `<empieza aqui pero termina en el primer > o en el último >` 
    
Despues de todo, ambos cumplen con lo expresado en la expresion regular: "Un signo de menor que, luego lo que sea y al final un simbolo de mayor que". Los informáticos odiamos la ambigüedad; en este caso se resuelve haciendo que por defecto, el buscador de la expresiñón regular intente darnos **la mayor cantidad posible de texto**. Esto se conoce como modo avaricioso o _Greedy_. Podemos indicar que queremos el comportamiento contrario, esto es, que nos de **la menos cantidad posible de texto** (o modo _non greedy_) añadiendo un caracter `?` despues del asterisco. Veamoslo en el siguiente ejemplo:

In [5]:
import re

pat_greedy = re.compile(r'<.*>')
pat_non_greedy = re.compile(r'<.*?>')

text = 'Hola <empieza aqui pero termina en el primer > o en el último >'

match = pat_greedy.search(text)
if match:
    print(f"En modo greedy encontró: {match.group(0)}")
match = pat_non_greedy.search(text)
if match:
    print(f"En modo non-greedy encontró: {match.group(0)}")
        

En modo greedy encontró: <empieza aqui pero termina en el primer > o en el último >
En modo non-greedy encontró: <empieza aqui pero termina en el primer >


### El caracter especial Suma o Más `+`

Similar a `*`, el caracter especial `+` se interpreta como "El patrón 
anterior, repetido **1** o más veces". Por ejemplo, el
patron `e+` **no** casaria con la cadena vacia (Ninguna aparición del caracter `e`, requerimos al menos una), pero si casaría con `e` (Una repetición de 'e'), con `ee` (Dos repeticiones), `eee` (Tres repeticiones), etc.

Una combinacion muy habitual es el patrón `.+`. Esto se leeria como "Una o mas repeticiones de la expresion regular que esta justo antes, que en este caso es _cualquier caracter_", o lo que el lo mismo, "cualquier caracter, repetido 1 o mas veces". Aun más resumido: "Todo, menos la cadena vacia".

De igual forma, por defecto se comporta en modo _greedy_ y se puede
cambiar a _non greedy_ con el sufijo `?`.

### El caracter especial interrogación `?`

El caracter especial `?` debe interpretarse como "El patron anterior, **0** o **1** vez". Por ejemplo, el patron `este?` casaria con la cadena `est` y con `este`. Otra forma de leerlo es "opcionalmente, puede venir el patron anterior".

### Los caracteres especiales Corchetes `[` y `]`

Estos caracteres se usan para definir un conjunto de caracteres, de forma que cualquiera de ellos
se acepta como una ocurrencia. 

En un conjunto Los caracteres se pueden listar individualmente, como por
ejemplo, `[abc]`, que casa con cualquiera de los caracteres `a`, `b` o `c`.

Por ejemplo `[aeiuo]` es un patron que se interpretaria como
"cualquier vocal". Otro uso muy frecuente sería `[0123456789]`, que se interpretarian como
"cualquier digito".

Se acepta tambien una forma abreviada que nos permite incluir un rango, usando el caracter `-`. Por ejemplo, el patrón anterior `[0123456789]` puede abreviarse como `[0-9]`. El patron `[0-9A-F]` casaria con
cualquier digito y con las letras `A`, `B`, `C`, `D`, `E` y `F`. 

Los caracteres especiales pierden su significado
dentro de los corchetes, por lo que no hace falta escaparlos.

Se puede definir el __complemento del conjunto__ incluyendo como primer
caracter `^`. De esta forma, la expresión regular `[^59]` casa con
cualquier caracter, excepto con los dígitos `5` y `9`.

### El caracter especial Barra Vertical o Tubería `|`

Se usa en la forma `A|B`, donde A y B representan expresiones regulares, y se interpretan
como una expresion regular que acepta cualquiera de las dos, es decir, que casará con cualquier texto que case con A o con B. Es muy habitual su uso con los grupos, que veremos más adelante.

Se pueden encadenar, por ejemplo, el siguiente patron:

    este|ese|aquel
    
casaria con cualquier de estas palabras.

In [ ]:
pat_psoe = re.compile('PSOE|Partido Socialista Obrero Español')

### Los caracteres especiales Llaves `{` y `}`

Estos caracteres nos permite definir el número de veces que se debe repetir la expresión regular precedente, o definir un rango de repeticiones válido.

Por ejemplo, `[0-9]{4}` se leeria "Cualquier dígito, repetido 4 veces", o esa que `3622` casa, pero ni `231` ni `56423` lo harían, porque le falta un dígito en el primer caso y sobra uno en el segundo.

Si cambiaramos el patron a `[0-9]{3,4}` se leeria "Cualquier dígito, repetido 3 o 4 veces", o esa que `3622` casa, y `231` también (pero `75` no, le falta un digito).

**Miniejercicio**: Modificar el patrón para que acepte 2, 3, o 4 dígitos. 

**Ejercicio**: Escribir el patron para encontrar posibles NIF: de 7 a 8 digitos seguidos de una letra muyuscula


- "43478329W" Correcto
- "434783294W"  Demasiados digitos
- "434783W" Pocos digitos
- "43783294"  Falta la letra
- "W33783294"  Letra en lugar incorrecto



### El caracter especial Barra Invertida `\`

El propósito de este caracter especial es doble: Si precede a otro caracter
especial, entonces reconvierte a dicho caracter de especial a normal (Se dice que
*escapa* el significado del caracter). Esto permite buscar caracteres como `*` o
`?` de forma literal.

Por ejemplo, la expresión r"`Doctor Who\?`" busca literalmente el texto `Doctor Who?`. Si no escapáramos la interrogación (Es decir, si se usara r"`Doctor Who?`"), se interpretaria como que la última `o` es opcional, y casaria, por ejemplo, con `Doctor Wh`, que no es lo que queremos.

El segundo uso es introducir una secuencia especial, que definiremos a continuación.

**Importante**: Recordemos que Python también usa el caracter '\' como su propia forma
de escapar significados (por ejemplo `\n` es la forma de representar un salto de línea). Asi
que para incluir la barra invertida tendriamos que escribirla dos veces. Es por eso que
siempre se recomiendo usar cadenas de texto "_raw_" (con una `r` antes de la primera comilla).

**A recordar**: definir __siempre__ las expresiones regulares usando cadenas crudas (*raw*). Algunos verificadores de código o _linters_ incluso disparan una alerta si no se hace.

### Secuencias especiales

Algunas de estas secuencias especiales accesible con `\` son:

- `\b` coincide con una cadena vacia, pero solo al principio de una palabra

- `\B` coincide con una cadena vacia, pero solo si __no__ esta al principio de una palabra. Esto
  significa qur r"py\B" casará con "python", "py3", "py2", pero no con "py", "py." o "py!". 
  `\B` es el opuesto de `\b`. Veremos esta pauta repetida en otras secuencias especiales.
  

  
- `\d`: casa con cualquier dígito. Equivalente a r"[0-9]"

- `\D`: casa con cualquier caracter que no sea un dígito. Equivale a r"[^0-9]"

  
- `\s`: Casa con espacios y equivalentes, como tabuladores, saltos de linea, etc.

- `\S`: Casa con cualquier cosa que no sean espacios. El opuesto de `\s`

- `\w`: Caracteres que pueden ser partes de una palabra en cualquier lenguaje, asi
  como los digitos del 0 al 9 y el caracter `_`. Equivale a r"[a-zA-Z0-9_]"

### Los caracteres especial Paréntesis `(` y `)` (Grupos)

Sirven para indicar el principio y el fin de un grupo. No modifican la exptresion regular,
en el sentido que esta sigue casando exactamente igual con parentesis o sin ellos, pero sirven para que podamos recuperar, despues de una coincidencia o *match*, los contenidos de estos grupos.

Por ejemplo, supongamos que queremos buscar por indicadores de tareas al estilo de Jira, que se forman
con la estructura: Código de proyecto seguido de guión y seguido del numero de tarea. Algunos indicadores validos podrian ser "ALPH-1244" o "BE-123". Supongamos para simplificar que los codigos de proyecto son siempre
letras mayúsculas, el patrón que detecta estos códigos podría ser:

    [A-Z]+-\d+
    

Comprobemos si funciona:

In [15]:
import re

patron = re.compile(r"[A-Z]+-\d+")
for codigo in ["ALPH-1244", "BEMAC-123", "MZGZ-1", "COVID-12"]:
    if patron.match(codigo):
        print(f"Codigo {codigo} es correcto")

Codigo ALPH-1244 es correcto
Codigo BEMAC-123 es correcto
Codigo MZGZ-1 es correcto
Codigo COVID-12 es correcto


Ahora, si quisieramos acceder al numero de tarea, la forma mas fácil
es usar los parentesis para crear los grupos que nos interesan. 

Para ello, cambiamos el patron de:

    [A-Z]+-\d+
    
a:

    [A-Z]+-(\d+)


Ahora, el patron se comporta exactamente igual que antes, pero los objetos *match* 
resultantes de una coincidencia permiten acceder a los grupos definidos con el método 
`group`, indicando el numero de orden de definición del grupo, siendo elg grupo 1 el primer
grupo definido.

Tambien se puede usar el grupo 0, y de hemos utilizado antes en los ejemplos. Este grupo cero esta definido siempre y consiste en la totalidad del texto que haya casado con el patrón. 

In [22]:
import re

patron = re.compile(r"([A-Z]+)-(\d+)")
for codigo in ["ALPH-1244", "BEMAC-123", "MZGZ-1", "COVID-12"]:
    m = patron.match(codigo)
    if m:
        proyecto, task_number = m.groups()
        task_number = int(task_number)
        print(f"{m.group(0)}: Tarea número {task_number} del proyecto {proyecto}")
        

ALPH-1244: Tarea número 1244 del proyecto ALPH
BEMAC-123: Tarea número 123 del proyecto BEMAC
MZGZ-1: Tarea número 1 del proyecto MZGZ
COVID-12: Tarea número 12 del proyecto COVID


**Miniejercicio**: modifica el código anterior para añadir otro grupo, esta
vez para capturar el código del proyecto. Incluirlo en el listado.

### El método split

Con este método podemos dividir una cadena de texto, usando una expresión
regular para determinas los puntos de corte. Los separadores en si no se devuelven, pero podemos forzar a que se incluyan ssi loas agrupamos con paréntesis.

El siguiente ejemplo muestra como podemos dividir un formato de fecha
que utiliza como separador tanto la barra normal inclinada `/`, la barra invertida `\`, la barra vertical `|` o el guión:


In [24]:
import re

pat_sep = re.compile(r"[-\\|/]")
print(pat_sep.split("2020-10-19"))
print(pat_sep.split("2020/10/19"))
print(pat_sep.split("2020\\10\\19"))
print(pat_sep.split("2020|10|19"))


['2020', '10', '19']
['2020', '10', '19']
['2020', '10', '19']
['2020', '10', '19']


## Ejercicio: Expresiones regulares para encontrar matrículas de coche.

Escribir una expresión regular para detectar matrículas de coches
españolas.

Según el siguiente texto, que describen en el sistema de matriculación
vigente actualmente en España:

> El 18 de septiembre del año 2000 entró en vigor el nuevo sistema de
> matriculación en españa, introduciendo matrículas que constan de cuatro
> dígitos y tres letras consonantes, suprimiéndose las cinco vocales y las
> letras Ñ, Q, CH y LL. \[...\] Si el vehículo es histórico, y se ha
> matriculado con una placa de nuevo formato, aparece primero una letra H
> en la placa.

El siguiente código lista las matrículas encontradas en el texto:

In [29]:
import re

Texto = '''INSTRUIDO por accidente de circulación ocurrido a las 09:43
entre la motocicleta HONDA 500, matrícula 0765-BBC  y la
motocicleta HARLEY-DAVIDSON , matrícula 9866-LPX, en el punto
kilométrico 3.5 de la carretera general del sur, término municipal de
Arona, Tenerife, y bla, bla, bla...'''


patron = re.compile(r"H?\d{4}-?[BCDFGHJKLMNPRSTVWXYZ]{3}")
for matricula in re.findall(patron, Texto):
    print(matricula)


0765-BBC
9866LPX


**Solución** 

El patrón usado es el siguiente:

    H?\d{4}-?[BCDFGHJKLMNPRSTVWXYZ]{3}
    
Es verdad que visto así puede asustar un poco, pero solo es cosa de verlo por
por partes:

- `H?` : Una `H`. Pero como la sigue una interrogación, es opcional.
    Recuerdese que `?` se interpreta como la expresión regular anterior
    (en este caso la H), 0 o 1 vez.

- `\d{4}` : El conjunto de los caracteres del `0` al `9` (`[0-9]`), o
    lo que es lo mismo, cualquier dígito, repetido 4 veces (`{4}`), es
    decir, un número de cuatro dígitos.

-  `-?` : Un guión, opcional, igual que la H para vehículos históricos
    del principio

-   `[BCDFGHJKLMNPRSTVWXYZ]`{3} : Cualquiera de los caracteres del
    conjunto indicado (letras consonantes excepto la Ñ, Q, CH y LL)
    repetido 3 veces.
    

Un trupo que podemos usar con las expresiones para que a la hora de escribirlas y leerlas
sean más sencillas es usar un parametro opcional a la hora de compilar, con el que podemos indicarle que, en el texto que define la expresion regular, se ignoren los espacios (a no ser que se escapen) y los saltos de linea, y que incluso podemos comentar la expresion regular con el caracter `#`, todo lo que escribamos a partir de este caracter y hasta el final de la linea será ignorado y no formara parte del patrón. 

Así, usando la constante `re.VERBOSE` como segundo parámetro, podemos escribir la expresión regular del ejercicio anterior como:

In [30]:
import re 

pat_matricula = re.compile("""
    H?  # La letra H, opcional, se reserva para vehículos históricos
    \d{4}  # Los cuatro dígitos de la matrícula
    -?  # Un guión, opcional. Es decir, aceptamos 0765-BBC o 0765BBC
    [BCDFGHJKLMNPRSTVWXYZ]{3}  # Las cuatro letras, del conjunto de posibles
    """, re.VERBOSE)

assert pat_matricula.match("0765-BBC")
assert pat_matricula.match("9866-LPX")
assert pat_matricula.match("probóscide") is None


### El método sub

Muchas veces, de lo que se trata es de buscar una texto que siga un patron y reemplazarlo por otro texto. Los opjetos `Pattern` tienen un método para realizar estos cambios de forma sencilla y potente. Em método `sub` necesita al menos dos parámetros, el primero es aquello que queremos poner como sustitucion de lo encontrado por el patrón, y luego el texto sobre el que ejecutar la transformación. Devuelve el texto tranformado.

Por ejemplo, el siguiente código reemplaza todas las vocales por asteriscos

In [35]:
import re

pat = re.compile(r"[aeiouáéíúóú]", re.IGNORECASE)
texto = "El lémur de cola Anillada (Lemur catta) es un gran prosimio"

print(pat.sub("_", texto))

_l l_m_r d_ c_l_ _n_ll_d_ (L_m_r c_tt_) _s _n gr_n pr_s_m__


Con lo que es muy fácil cambiar un texto a modo burleta:

In [43]:
import re
import random

pat = re.compile(r"[aeiou]")
texto = "Te he dicho mil veces que no me remedes"

def upper(match):
    return random.choice('aeiou')

print(pat.sub(upper, texto))

Ta hu decho mol vaces qua nu mu remudos


Si el primer parámetro es un texto, se substituye sin más. Pero podemos
hacer cambios aun más potentes, si en vez de pasar un texto, pasamos una función.
Esa función debera aceptar un parámetro, que será el objeto _Match_ o
coincidencia encotrada, y en base a el hacer el cambio oportuno.

Por ejemplo, el  siguiente ejemplo busca números en coma flotante y
los reemplaza por la fracción equivalente, usando la clase `Fraction` que vimos
del módulo `fractions`:

In [46]:
from fractions import Fraction

Fraction(0.25)

Fraction(1, 4)

In [53]:
import re
from fractions import Fraction

pat_decimal = re.compile('\d+\.\d+')

def as_fracciones(match):
    num = float(match.group(0))
    f = Fraction(num)
    if f.denominator == 1:
        return str(f.numerator)
    else:
        return f"{f.numerator}/{f.denominator}"

text = f"La suma de 0.25 y 0.25 da como resultado {0.25+0.25}"
print(pat_decimal.sub(as_fracciones, text))

La suma de 1/4 y 1/4 da como resultado 1/2


Podemos, por tanto, realizar cambios dinámicos, basándonos en los textos encontrado o en cualquier otro dato. En el siguiente ejemplo, más complejo, se estraen y numeran las notas que encuentre un texto, suponiendo que todas las notas siguen el format `NOTA: <lo que sea>.`:

In [54]:
import re

pat_nota = re.compile('NOTA: (.+?)\.', re.DOTALL|re.MULTILINE)

counter = 0
notas = []
def numera_notas(match):
    global counter, notas
    counter += 1
    texto_nota = match.group(1).replace('\n', ' ').capitalize()
    notas.append(texto_nota)
    return f"(Véase Nota #{counter})."

text = """
Indiana es uno de los cincuenta estados de los Estados Unidos NOTA: localizado
en la región del Medio Oeste (Midwest) del país. Su capital es 
Indianápolis NOTA: su población es de 829 718 hab. Limita al norte con el lago 
y el estado de Míchigan, al sur con Kentucky, al este con Ohio y
con Illinois por el oeste NOTA: cubierta en su mayor parte por llanuras. La 
palabra Indiana significa «tierras de los indios».
"""
print(pat_nota.sub(numera_notas, text))
for num, nota in enumerate(notas, start=1):
    print(f"- Nota {num}: {nota}")


Indiana es uno de los cincuenta estados de los Estados Unidos (Véase Nota #1). Su capital es 
Indianápolis (Véase Nota #2). Limita al norte con el lago 
y el estado de Míchigan, al sur con Kentucky, al este con Ohio y
con Illinois por el oeste (Véase Nota #3). La 
palabra Indiana significa «tierras de los indios».

- Nota 1: Localizado en la región del medio oeste (midwest) del país
- Nota 2: Su población es de 829 718 hab
- Nota 3: Cubierta en su mayor parte por llanuras


**Ejercicio**: Buscar en la [documentación oficial de Python del módulo `re`](https://docs.python.org/es/3/library/re.html#re.MULTILINE) el significado 
de las opciones `DOTALL` y `MULTILINE`.

**Pregunta**  ¿Qué hace el siguiente programa?

In [56]:
import os
import re

pat_notebook = re.compile("^[abc].*\.ipynb$")
for (dir_path, directories, files) in os.walk("../.."):
    for fn in files:
        if pat_notebook.search(fn):
            print(fn)
    

arrow.ipynb
arrow-checkpoint.ipynb
compression.ipynb
argparse.ipynb
argparse-checkpoint.ipynb
base64-checkpoint.ipynb
argparse-checkpoint.ipynb
collections-checkpoint.ipynb
compression-checkpoint.ipynb
csv-checkpoint.ipynb
collections.ipynb
collections-checkpoint.ipynb
base64.ipynb
base64-checkpoint.ipynb
csv.ipynb
csv-checkpoint.ipynb


Hay más cosas que podemos hacer con el módulo `re`, consulta la [documentación oficial sobre el módulo re](https://docs.python.org/es/3/library/re.html).